In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [7]:
# Problem parameters
sigma_s = 0 # Macroscopic cross section for scattering [cm^(-1)] {Play with sigma_s = ln(2) or something}
sigma_t = 1 # Macroscopic total interaction cross section [cm^(-1)] (reciprocal is neutron mean free path (mfp))
q_ext = 3 # External source density rate [TODO: insert units]
# q = q_ext/2.0 # Source density rate [TODO: insert units]

# Left- and right-endpoints of domain
a = 0
b = 10

# Number of cells in domain
ncells = 9
# Number of points in domain
npts   = ncells + 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Auxillary term for calculation clarity in the loops below
aux_term = (sigma_t/2.0)*dx

# BCs
psi_left = 0.5
psi_right = 1.5

# Quadrature order (needs to be EVEN to avoid mu = 0)
m = 16
angles, weights = scipy.special.roots_legendre(m)
# By default, angles go from negative to positive, and we want the opposite
angles = np.flip(angles)

# Array to store angular fluxes in all directions at the previous cell interface
psi_prev = np.full((m, ), psi_right) # Enforce right BC
psi_prev[0::m//2] = psi_left # Enforce left BC

# Array to store scalar fluxes at the midpoint of each cell
sca_fluxes = np.full((ncells, ), np.log(2)) # Initial guesses for midpoint scalar fluxes

# If the absolute difference of successive iterates of the scalar flux at
# cell interface i is below this, then we stop updating the scalar flux at 
# that interface
eps = 1e-14

for i in np.arange(0, ncells):
    next_sca_flux = 0.0
    sca_flux_err = np.abs(sca_fluxes[i] - next_sca_flux)
    
    # Initial guess for the source term at midpoint of cell i
    curr_source = (q_ext/2.0) + (sigma_s/2.0)*sca_fluxes[i]

    while sca_flux_err >= eps: 
        for j, mu in enumerate(angles):
            abs_mu = np.abs(mu) 
            psi_next_i_j = ((abs_mu - aux_term)*psi_prev[j] + curr_source*dx) / \
                            (abs_mu + aux_term)

            # Update scalar flux at cell interface i using the average of the
            # angular flux at the previous cell interace and the just computed angular
            # flux at the current cell interface 
            psi_avg_i_j            = (psi_next_i_j + psi_prev[j])/2.0
            next_sca_flux   += psi_avg_i_j * weights[j]

            # If the difference is small enough, set the angular flux at the 
            # previous cell interface to be the angular flux at the current interface
            if sca_flux_err < eps:
                psi_prev[j] = psi_next_i_j

        sca_flux_err = np.abs(next_sca_flux - sca_fluxes[i])

        # Update the source term
        sca_fluxes[i] = next_sca_flux
        curr_source    = (q_ext/2.0) + (sigma_s/2.0)*sca_fluxes[i]
        next_sca_flux  = 0.0

In [8]:
print(sca_fluxes)

[2.56933648 2.56933648 2.56933648 2.56933648 2.56933648 2.56933648
 2.56933648 2.56933648 2.56933648]


In [9]:
# Problem parameters
sigma_s = 0.5 # Macroscopic cross section for scattering [cm^(-1)] {Play with sigma_s = ln(2) or something}
sigma_t = 1 # Macroscopic total interaction cross section [cm^(-1)] (reciprocal is neutron mean free path (mfp))
q_ext = 3 # External source density rate [TODO: insert units]
# q = q_ext/2.0 # Source density rate [TODO: insert units]

# Left- and right-endpoints of domain
a = 0
b = 10

# Number of cells in domain
ncells = 9
# Number of points in domain
npts   = ncells + 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Auxillary term for calculation clarity in the loops below
aux_term = (sigma_t/2.0)*dx

# BCs
psi_left = 1.5
psi_right = 1.5

# Quadrature order (needs to be EVEN to avoid mu = 0)
m = 16
angles, weights = scipy.special.roots_legendre(m)
# By default, angles go from negative to positive, and we want the opposite
angles = np.flip(angles)

eps = 1e-14
psi_prev = np.full((m, ), psi_right) # Enforce right BC
psi_prev[0::m//2] = psi_left         # Enforce left BC

curr_scalar_fluxes = np.full((ncells, ), np.log(2))
new_scalar_fluxes  = np.zeros(curr_scalar_fluxes.shape)
scalar_flux_err    = np.abs(new_scalar_fluxes - curr_scalar_fluxes)

while np.any(scalar_flux_err >= eps):
    curr_source_term = (q_ext + sigma_s*curr_scalar_fluxes)/2.0

    for d, mu in enumerate(angles):
        # print(f"{d} = {psi_prev[d]}")
        abs_mu = np.abs(mu)
        
        for i in np.arange(ncells):
            psi_d_i_next      = ((abs_mu - aux_term) * psi_prev[d] + curr_source_term[i]*dx)/ \
                                 (abs_mu + aux_term)
            avg_psi           = (psi_d_i_next + psi_prev[d])/2.0
            new_scalar_fluxes[i] += avg_psi * weights[d]

        # Update the previous angular flux for the next direction mu
        psi_prev[d] = psi_d_i_next
    
    # Update source term with updated scalar flux
    scalar_flux_err = np.abs(new_scalar_fluxes - curr_scalar_fluxes)
    curr_scalar_fluxes = new_scalar_fluxes
    new_scalar_fluxes[:] = 0.0

0 = 1.5
1 = 1.5
2 = 1.5
3 = 1.5
4 = 1.5
5 = 1.5
6 = 1.5
7 = 1.5
8 = 1.5
9 = 1.5
10 = 1.5
11 = 1.5
12 = 1.5
13 = 1.5
14 = 1.5
15 = 1.5
0 = 1.6246254406948886
1 = 1.6283494158654297
2 = 1.6354789456154952
3 = 1.6468701476699026
4 = 1.6640835739203017
5 = 1.6899626471438551
6 = 1.7299931781568278
7 = 1.7959580922136182
8 = 1.7959580922136182
9 = 1.7299931781568278
10 = 1.6899626471438551
11 = 1.6640835739203017
12 = 1.6468701476699026
13 = 1.6354789456154952
14 = 1.6283494158654297
15 = 1.6246254406948886


In [7]:
print(curr_scalar_fluxes)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]
